# Table of Contents
 <p><div class="lev1"><a href="#Initialization-Cells"><span class="toc-item-num">1 - </span>Initialization Cells</a></div><div class="lev1"><a href="#Gather-Data-from-Big-Query-GDELT"><span class="toc-item-num">2 - </span>Gather Data from Big Query GDELT</a></div><div class="lev2"><a href="#Original-Queries"><span class="toc-item-num">2.1 - </span>Original Queries</a></div><div class="lev1"><a href="#New-Queries---Multiple-Years-and-Quarter"><span class="toc-item-num">3 - </span>New Queries - Multiple Years and Quarter</a></div><div class="lev2"><a href="#Execute-Queries"><span class="toc-item-num">3.1 - </span>Execute Queries</a></div>

# Initialization Cells

In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:

import json
import os
import pandas as pd
from shapely.geometry import shape, Point

In [3]:
from geo import geo

# Gather Data from Big Query GDELT

In [4]:
# Imports the Google Cloud client library
from google.cloud import bigquery
from pandas.io import gbq
import pandas as pd


#gbq._authenticate = lambda: client
# Instantiates a client
bigquery_client = bigquery.Client()


## Original Queries


In [76]:
# political event counts and average 
query_pol = """SELECT count(*),  count(*) / count(distinct(SQLDATE)),ActionGeo_ADM1Code
FROM [gdelt-bq:gdeltv2.events]
WHERE IsRootEvent=1
AND SQLDATE>20170601 AND SQLDATE<20170901
AND ActionGeo_CountryCode='BR'
AND (EventCode='1721' OR EventCode='1722' OR EventCode='1723' OR EventCode='1724' OR EventBaseCode='173' OR EventBaseCode='174' OR EventBaseCode='182' OR EventBaseCode='183')
GROUP BY ActionGeo_ADM1Code, 
ORDER BY ActionGeo_ADM1Code
"""

In [103]:
# ESG event counts and average
query_esg = """SELECT count(*),  count(*) / count(distinct(SQLDATE)),ActionGeo_ADM1Code
FROM [gdelt-bq:gdeltv2.events]
WHERE IsRootEvent=1
AND SQLDATE>20170101 AND SQLDATE<20170901
AND ActionGeo_CountryCode='BR'
AND (Actor1Type1Code='ENV' OR Actor1Type2Code= 'ENV' OR Actor1Type3Code='ENV' 
OR Actor1Type1Code='LAB' OR Actor1Type2Code= 'LAB' OR Actor1Type3Code='LAB' OR 
Actor1KnownGroupCode='IGOUNOLABILO' OR Actor1KnownGroupCode='IGOUNOLABILO' OR Actor1KnownGroupCode='NGOIHF')
GROUP BY ActionGeo_ADM1Code
ORDER BY ActionGeo_ADM1Code
"""

# New Queries - Multiple Years and Quarter

In [5]:
# political event counts and average 
query_pol = """SELECT count(*),  count(*) / count(distinct(SQLDATE)),ActionGeo_ADM1Code, Quarter(DATE(CAST(SQLDATE as STRING))) as q, Year(DATE(CAST(SQLDATE as STRING))) as y
FROM [gdelt-bq:gdeltv2.events]
WHERE IsRootEvent=1
AND  Year > 2014
AND ActionGeo_CountryCode='BR'
AND (EventCode='1721' OR EventCode='1722' OR EventCode='1723' OR EventCode='1724' OR EventBaseCode='173' OR EventBaseCode='174' OR EventBaseCode='182' OR EventBaseCode='183')
GROUP BY ActionGeo_ADM1Code,y,q
ORDER BY ActionGeo_ADM1Code, y, q
"""

In [6]:
# ESG event counts and average
query_esg = """SELECT count(*),  count(*) / count(distinct(SQLDATE)),ActionGeo_ADM1Code, Quarter(DATE(CAST(SQLDATE as STRING))) as q, Year(DATE(CAST(SQLDATE as STRING))) as y
FROM [gdelt-bq:gdeltv2.events]
WHERE IsRootEvent=1
AND Year > 2014
AND ActionGeo_CountryCode='BR'
AND (Actor1Type1Code='ENV' OR Actor1Type2Code= 'ENV' OR Actor1Type3Code='ENV' 
OR Actor1Type1Code='LAB' OR Actor1Type2Code= 'LAB' OR Actor1Type3Code='LAB' OR 
Actor1KnownGroupCode='IGOUNOLABILO' OR Actor1KnownGroupCode='IGOUNOLABILO' OR Actor1KnownGroupCode='NGOIHF')
GROUP BY ActionGeo_ADM1Code,y,q
ORDER BY ActionGeo_ADM1Code, y, q
"""

## Execute Queries

In [7]:
results_pol = gbq.read_gbq(query_pol, project_id='expedition-hacks-morocco')
results_pol.columns = ['pol count', 'pol average', 'Adm1Code', 'quarter', 'year']
results_esg = gbq.read_gbq(query_esg, project_id='expedition-hacks-morocco')
results_esg.columns = ['esg count', 'esg average', 'Adm1Code','quarter', 'year']


URL being requested: POST https://www.googleapis.com/bigquery/v2/projects/expedition-hacks-morocco/jobs?alt=json
Making request: POST https://accounts.google.com/o/oauth2/token


Requesting query... 

URL being requested: POST https://www.googleapis.com/bigquery/v2/projects/expedition-hacks-morocco/jobs?alt=json


ok.
Job ID: job_ZhrjPLVraq1DW3taln7cjEReF-0M
Query running...


URL being requested: GET https://www.googleapis.com/bigquery/v2/projects/expedition-hacks-morocco/queries/job_ZhrjPLVraq1DW3taln7cjEReF-0M?alt=json


...

Requesting query... 

URL being requested: POST https://www.googleapis.com/bigquery/v2/projects/expedition-hacks-morocco/jobs?alt=json


ok.
Job ID: job_1NkIfS3iyRmqeRHc31XZKlGwtHaI
Query running...


URL being requested: GET https://www.googleapis.com/bigquery/v2/projects/expedition-hacks-morocco/queries/job_1NkIfS3iyRmqeRHc31XZKlGwtHaI?alt=json


Query done.
Processed: 9.5 GB
Standard price: $0.05 USD

Retrieving results...
Got 316 rows.

Total time taken 1.85 s.
Finished at 2017-10-14 02:24:02.


In [8]:
results_pol.to_csv('brazil_results_pol.csv')
results_esg.to_csv('brazil_results_esg.csv')

In [9]:
results_pol.head()

,pol count,pol average,Adm1Code,quarter,year
0,926,17.807692,BR,1,2015
1,1512,16.615385,BR,2,2015
2,1383,15.032609,BR,3,2015
3,1448,15.739130,BR,4,2015
4,2005,22.032967,BR,1,2016


In [10]:
results_esg.head()

,esg count,esg average,Adm1Code,quarter,year
0,260,6.190476,BR,1,2015
1,739,8.211111,BR,2,2015
2,714,7.933333,BR,3,2015
3,797,8.955056,BR,4,2015
4,759,8.340659,BR,1,2016
